# Step 1 - Prepare network data
## Project: Growing Urban Bicycle Networks

This notebook downloads bicycle and street network data sets and prepares them for analysis.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-06-09  
Last modified: 2021-03-10

## Preliminaries

### Parameters

In [1]:
debug = True # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

Loaded parameters.



### Setup

In [2]:
%run -i path.py
#%run -i setup.py
%run -i setupCPH.py

%load_ext watermark
%watermark -n -v -m -g -iv
import statistics

Loaded PATH.

Setup finished.

Python implementation: CPython
Python version       : 3.8.2
IPython version      : 8.5.0

Compiler    : Clang 9.0.1 
OS          : Darwin
Release     : 18.7.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Git hash: b795eaefba9eb6349c319ca07997ebd31e033f57

osmnx     : 0.16.2
osgeo     : 3.2.1
networkx  : 2.8.6
pyproj    : 3.4.0
geojson   : 2.5.0
numpy     : 1.23.3
csv       : 1.0
sys       : 3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 07:56:27) 
[Clang 9.0.1 ]
watermark : 2.3.1
pandas    : 1.4.4
igraph    : 0.9.1
fiona     : 1.8.21
geopandas : 0.11.1
matplotlib: 3.6.0
shapely   : 1.8.4



### Functions

In [3]:
%run -i functions.py

Loaded functions.



## Load carall graph

In [4]:
G_carall = ox.load_graphml('../../bikenwgrowth_external/data/copenhagen/bikedata/G_carall_graphml.graphml')

# Set alpha

In [5]:
a = 0.5

## Add 'bcount_pop_den_attr' to carall graph

In [ ]:
nx.set_edge_attributes(G_carall, 0.0, 'bcount_pop_den_attr')

for e in G_carall.edges():
    a,b=e
    bcount_attr = G_carall.get_edge_data(a,b,0)['bcount_attr']
    pop_den_attr = G_carall.get_edge_data(a,b,0)['pop_den_attr']
    bcount_pop_den_attr = a* bcount_attr + (1-a)*pop_den_attr
    nx.set_edge_attributes(G_carall, {(a,b,0): {"bcount_pop_den_attr": bcount_pop_den_attr }})

    
# a dictionary of each of the two new attributes in the G_carall graph is generated and saved. 
bcount_pop_den_attr_dict = nx.get_edge_attributes(G_carall, "bcount_pop_den_attr")
print(bcount_pop_den_attr_dict)

In [ ]:
ox_to_csv(G_carall, PATH["data"] + 'copenhagen' + "/", 'copenhagen', 'carall')

ox_to_csv(ox.simplify_graph(G_carall), PATH["data"] + 'copenhagen' + "/", 'copenhagen', 'carall', "_simplified")

ox.save_graphml(G_carall, "../../bikenwgrowth_external/data/copenhagen/bikedata/G_carall_graphml.graphml")

with open('../../bikenwgrowth_external/data/copenhagen/bikedata/edges_dict_bcount_pop_den_attr.pkl', 'wb') as f:
    pickle.dump(bcount_pop_den_attr_dict, f)

In [ ]:
Audio(sound_file, autoplay=True)